## most_popular 만들기 (첨부한 파일 그대로 사용하면 실행 안해도됩니다!) : 베이스라인코드

### 파이참에서 베이스라인코드양식그대로 사용하여 나온 results를 most_pupular로이름을 변경한 후 저장하여 사용했습니다. ( 곡을전혀 뽑지 못하는 플레이리스트에 적용하였습니다.)

In [ ]:
python genre_most_popular.py run song_meta.json train.json test.json

## 토크나이징을 위한 khaiii설치

In [33]:

!git clone https://github.com/kakao/khaiii.git
!cd /content/khaiii/ && mkdir build
!cd /content/khaiii/build && pip install cmake
!cd /content/khaiii/build && cmake /content/khaiii/build
!cd /content/khaiii/build && cmake ..
!cd /content/khaiii/build/ && make all
!cd /content/khaiii/build/ && make resource
!cd /content/khaiii/build/ && make install
!cd /content/khaiii/build/ && make package_python
!cd /content/khaiii/build/package_python && pip install .


fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: 02ccb06 | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

### 설치 후에 첨부한 manual을 khaiii/rsc/src/ 에 preanal.manual 파일을 넣은후 아래 코드를 실행한다. 

In [34]:
!cd /content/khaiii/build/ && make all
!cd /content/khaiii/build/ && make resource
!cd /content/khaiii/build/ && make install
!cd /content/khaiii/build/ && make package_python
!cd /content/khaiii/build/package_python && pip install .

[ 65%] Built target obj_khaiii
[ 69%] Built target khaiii
[ 76%] Built target bin_khaiii
[100%] Built target test_khaiii
INFO:root:preanal.manual
INFO:root:preanal.auto
INFO:root:trie saved: /content/khaiii/build/share/khaiii/preanal.tri
INFO:root:total nodes: 121165
INFO:root:expected size: 1938640
INFO:root:value saved: /content/khaiii/build/share/khaiii/preanal.val
INFO:root:total entries: 83523
INFO:root:expected size: 594956
Built target resource
[ 65%] Built target obj_khaiii
[ 69%] Built target khaiii
[ 76%] Built target bin_khaiii
[100%] Built target test_khaiii
Install the project...
-- Install configuration: ""
-- Up-to-date: /usr/local/include/khaiii
-- Up-to-date: /usr/local/include/khaiii/khaiii_api.h
-- Up-to-date: /usr/local/include/khaiii/KhaiiiApi.hpp
-- Up-to-date: /usr/local/include/khaiii/khaiii_dev.h
-- Up-to-date: /usr/local/share/khaiii
-- Up-to-date: /usr/local/share/khaiii/hdn2tag.lin
-- Up-to-date: /usr/local/share/khaiii/errpatch.val
-- Up-to-date: /usr/local

## 드라이브 마운트 및 모델 실행을 위한 플레이리스트데이터 , 라이브러리 불러오기

In [35]:
##드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd

In [38]:
## 파일 불러오기
train = pd.read_json('/content/drive/My Drive/arena_data/train.json') 
val = pd.read_json('/content/drive/My Drive/arena_data/val.json') 
test = pd.read_json('/content/drive/My Drive/arena_data/test.json') 
song_meta = pd.read_json("/content/drive/My Drive/arena_data/song_meta.json")


In [39]:
train_for_popular = load_json('/content/drive/My Drive/arena_data/train.json')

In [40]:
# 인기곡)
_, song_mp = most_popular(train_for_popular, "songs", 200)

In [41]:
import json
import io
import re
import os
import distutils.dir_util
import numpy as np
import scipy.sparse as spr
import pickle
from collections import Counter

from gensim.models import Word2Vec
from gensim.models.keyedvectors import *
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
def remove_seen(seen, l):
    seen = set(seen)
    return [x for x in l if not (x in seen)]

def write_json(data, fname):
    def _conv(o):
        if isinstance(o, np.int64) or isinstance(o, np.int32):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname)
    distutils.dir_util.mkpath("./arena_data/" + parent)
    with io.open("./arena_data/" + fname, "w", encoding="utf8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)


def load_json(fname):
    with open(fname, encoding='utf8') as f:
        json_obj = json.load(f)

    return json_obj


def debug_json(r):
    print(json.dumps(r, ensure_ascii=False, indent=4))

def most_popular(playlists, col, topk_count):
    c = Counter()

    for doc in playlists:
        c.update(doc[col])

    topk = c.most_common(topk_count)
    return c, [k for k, v in topk]

class CustomEvaluator:
    def _idcg(self, l):
        return sum((1.0 / np.log(i + 2) for i in range(l)))

    def __init__(self):
        self._idcgs = [self._idcg(i) for i in range(101)]

    def _ndcg(self, gt, rec):
        dcg = 0.0
        for i, r in enumerate(rec):
            if r in gt:
                dcg += 1.0 / np.log(i + 2)

        return dcg / self._idcgs[len(gt)]

    def _eval(self, gt_fname, rec_fname):
        gt_playlists = load_json(gt_fname)
        gt_dict = {g["id"]: g for g in gt_playlists}
        rec_playlists = load_json(rec_fname)
        
        music_ndcg = 0.0
        tag_ndcg = 0.0

        for rec in rec_playlists:
            gt = gt_dict[rec["id"]]
            music_ndcg += self._ndcg(gt["songs"], rec["songs"][:100])
            tag_ndcg += self._ndcg(gt["tags"], rec["tags"][:10])

        music_ndcg = music_ndcg / len(rec_playlists)
        tag_ndcg = tag_ndcg / len(rec_playlists)
        score = music_ndcg * 0.85 + tag_ndcg * 0.15

        return music_ndcg, tag_ndcg, score

    def evaluate(self, gt_fname, rec_fname):
        try:
            music_ndcg, tag_ndcg, score = self._eval(gt_fname, rec_fname)
            print(f"Music nDCG: {music_ndcg:.6}")
            print(f"Tag nDCG: {tag_ndcg:.6}")
            print(f"Score: {score:.6}")
        except Exception as e:
            print(e)



## --------------------------------------------형태소 분석기관련 라이브러리
from typing import *
import matplotlib.pyplot as plt; plt.rcdefaults()
from khaiii import KhaiiiApi  # khaiii 레포는 https://github.com/kakao/khaiii 이쪽
tokenizer = KhaiiiApi()
import multiprocessing
# from gensim.models import FastText

# from gensim.models.fasttext import FastText as FT_gensim



## 플레이리스트 데이터 토크나이징 하기


In [42]:
def re_sub(series: pd.Series) -> pd.Series:
    series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True)  # ㅋ 제거용
    series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True)  # 특수문자 제거
    series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True)  # 공백 제거
    series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True)  # u3000 제거
    return series

def flatten(list_of_list : List) -> List:
    flatten = [j for i in list_of_list for j in i]
    return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:
  # -> 어노테이션을 뜻하며 리턴할떄 튜플이 return된다는걸로 일단 추정하고 넘어간다
  if len(title)== 0 or title== ' ':  # 제목이 공백인 경우 tokenizer에러 발생
      return []
  result = tokenizer.analyze(title)

  result_token = []
  for split in result:
    for morph in split.morphs:
      if len(morph.lex) > 1:
        result_token.append((morph.lex, morph.tag))
      elif len(morph.lex) == 1:
          # 원래 한글자단어는 다 삭제시키는데 특성으로 사용할만한 한글자 단어만 추출해서 사용하도록함
        if morph.lex == "비" or morph.lex == "밤" or morph.lex == "록" or morph.lex == "락"or morph.lex == "봄"or morph.lex == "춤"or morph.lex == "팝" or morph.lex == "넬" or morph.lex == "책":
          result_token.append((morph.lex, morph.tag))
  return result_token


def get_all_tags(df) -> List:
    tag_list = df['tags'].values.tolist() 
    tag_list = flatten(tag_list)
    return tag_list

def get_gns(target_song):
    if len(song_and_gn) <= 0:
        # song_and_gnd은 곡의 아이디 = 대분류 장르로 매핑되어있는 오브젝트이다. {곡번호:장르 ,곡번호:장르 ,}로 되어있음
        # for song in song_meta: 은 곡의 아이디와 장르를 새로운 오브젝트로 매핑하는 코드
        for song in song_meta:#함수화 하지않을거면 이부분 따로 전역으로 뺴서 써도됨.
            #밑의 if문은 장르를 한글로 집어넣기위한 예외처리 
            song_and_gn[song['id']] = song['song_gn_gnr_basket']
    res = song_and_gn.get(int(target_song))
    return res


with open(os.path.join('/content/drive/My Drive/song_meta.json'), encoding="utf-8") as f:
    song_meta = json.load(f)
song_and_gn = {}
def _get_gns(target_song):
    if len(song_and_gn) <= 0:
        # song_and_gnd은 곡의 아이디 = 대분류 장르로 매핑되어있는 오브젝트이다. {곡번호:장르 ,곡번호:장르 ,}로 되어있음
        # for song in song_meta: 은 곡의 아이디와 장르를 새로운 오브젝트로 매핑하는 코드
        for song in song_meta:#함수화 하지않을거면 이부분 따로 전역으로 뺴서 써도됨.
            song_and_gn[song['id']] = song['song_gn_gnr_basket']
    res = song_and_gn.get(int(target_song))
    return res
# print(_get_gns(13542)) >>>>  ['GN2500', 'GN0200']


In [43]:
using_pos = ['NNG','SL','NNP','XR']  # 일반 명사, 외국어, 고유 명사, 어근  
no_use_items = ['노래' , '플레이리스트' , '음악' ,'주관','요즘', '모음' , '나를' , '나만' ,'나에' , '내맘', '가요' , '초급' , '함께' , '너무' , '좋은' , '테스트' ,'가장' , '최고' , '명곡'
,'진짜' , '감성' , '매력' , '수록곡' , '그냥' , '없이' , '후기' , '느낌' , '오프닝' , '분위기', '삽입곡' ,'강추' ,'일때', '발매곡' , '입문' , '다재다능' ,'부제하',
'자외' , '비트' , '고고씽' , '디편','사이','특별','순간','완전','BUT','세상','자국','저릿','THE','리메이','얘기','개인','선곡','코로','추천곡','대변','사람','멜로디'
'방법', '질때' , '분위기','정규앨범','가사','아실테죠','입장','위주','극복','사람','연습곡','은곡' , '컬렉션','TOP','베스트','드루','모두','이번']

val['plylst_title'] = re_sub(val['plylst_title'])
val.loc[:, 'ply_token'] = val['plylst_title'].map(lambda x: get_token(x, tokenizer))
val['ply_token'] = val['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
val['ply_token'] = val['ply_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
val['ply_token'] = val['ply_token'].map(lambda x : ["_"+i[0] for i in x] )

test['plylst_title'] = re_sub(test['plylst_title'])
test.loc[:, 'ply_token'] = test['plylst_title'].map(lambda x: get_token(x, tokenizer))
test['ply_token'] = test['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
test['ply_token'] = test['ply_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
test['ply_token'] = test['ply_token'].map(lambda x : ["_"+i[0] for i in x] )

train['plylst_title'] = re_sub(train['plylst_title'])
train.loc[:, 'ply_token'] = train['plylst_title'].map(lambda x: get_token(x, tokenizer))
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
train['ply_token'] = train['ply_token'].map(lambda x : ["_"+i[0] for i in x] )

### 변환한 데이터를 저장

In [44]:
res = []
for idx in tqdm(val.index):
    id = val.loc[idx,"id"]
    songs = val[val["id"]==id]["songs"].values[0]
    tag = val[val["id"]==id]["tags"].values[0]
    token = val[val["id"]==id]["ply_token"].values[0]
        
    res.append({
            "id": id,
                "songs": songs,
                "tags": tag,
            "ply_token":token
         })
write_json(res, "val_token.json")
res = []
for idx in tqdm(train.index):
    id = train.loc[idx,"id"]
    songs = train[train["id"]==id]["songs"].values[0]
    tag = train[train["id"]==id]["tags"].values[0]
    token = train[train["id"]==id]["ply_token"].values[0]   
    res.append({
            "id": id,
                "songs": songs,
                "tags": tag,
                "ply_token":token
         })
##합쳐진 18000개의 VAL 저장
write_json(res, "train_token.json")
res = []
for idx in tqdm(test.index):
    id = test.loc[idx,"id"]
    songs = test[test["id"]==id]["songs"].values[0]
    tag = test[test["id"]==id]["tags"].values[0]
    token = test[test["id"]==id]["ply_token"].values[0]    
    res.append({
            "id": id,
                "songs": songs,
                "tags": tag,
                "ply_token":token
         })
write_json(res, "test_token.json")

100%|██████████| 10740/10740 [00:25<00:00, 419.96it/s]


## 토크나이징한 작업물을 word2vec을 이용하여 임베딩하기


### 임베딩작업후 나오는 모델은  총 2가지


#### 곡과 태그 , 플레이리스트 제목 토큰화화여  임베딩하는 모델

In [47]:
error2 = []
error= 0

In [48]:
class PlaylistEmbedding:
    ## init 에서 train , val 데이터를 주고 , Word2vec사용시 선택값들을 넣어준다.
    def __init__(self, FILE_PATH):
        with open(os.path.join(FILE_PATH, 'arena_data/train_token.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        with open(os.path.join(FILE_PATH, 'arena_data/val_token.json'), encoding="utf-8") as f:
            self.val = json.load(f)
        with open(os.path.join(FILE_PATH, 'arena_data/test_token.json'), encoding="utf-8") as f:
            self.test = json.load(f)
        self.most_results = pd.read_json("arena_data/most_popular.json")


        self.FILE_PATH = FILE_PATH  
        self.min_count = 2
        self.size = 128
        self.window = 210
        self.sg = 5
        self.p2v_model = WordEmbeddingsKeyedVectors(128)
       
        
        
      
            
    ## train 데이터와 val 데이터를 합친 후 그 플레이리스트의 id값에 곡들을 매칭시킨다.
  
    def get_dic(self, train, val,test):
        song_dic = {}
        tag_dic = {}
        data = train+val+test
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        ## 플레이리스트마다 곡과 태그들을 합쳐 리스트화 하는데 합친 값이 0 즉 곡과 태그들이 전혀없는경우는 제외한다.
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags'])+ list(x['ply_token']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
        #Word2vec메소드 실행 (곡과 태그를 함께 임베딩)
    def get_w2v(self, total, min_count, size, window, sg):
        print("학습시작")
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg,workers=6)
        w2v_model.wv.save_word2vec_format('token_model_send_dimension128_token') # 모델 저장
        # loaded_model = KeyedVectors.load_word2vec_format("token_model_send_dimension128_token")
        # w2v_model = loaded_model
        self.w2v_model = w2v_model
        print("학습완료")
    ## 
    def update_p2v(self, train, val,test, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train+val+test):
            tmp_vec = 0
            
            ##곡이 1개 이상인 경우에 곡과 태그의 백터값을 구한후 모두 더해 플레이리스트의 벡터,
            ##즉 플레이리스트의 백터를 구한다 (tmp_vec)
            ##여기서 나는 곡이 1개이상인  val플레이리스트기준으로 하였기때문에 왠만하면 if문 무조건넘어감
            for song in q['songs']+q['tags']+q['ply_token']:
                        
                    try:
                        
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                        not_error.append(str(song))
                    except:
                        
                        error2.append(str(song))
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
       #구한 플레이리스트 백터리스트를 p2vmodel에 저장한다.
        self.p2v_model.add(ID, vec)
        print("업데이트 완료")
    
    ## 플레이리스트 유사도를 구한후 결과를 내는 메소드
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []
        error = 0
        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                #문제플레이리스트와 가장 유사한 플레이리스트 60개의 ID를 뽑는다.

                most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=60)]

                get_song = []
                get_tag = []
                #플레이리스트ID를 이용하여 안에 들어있는 곡과 태그를 전부 꺼낸다.
                for ID in most_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                 
                #꺼내진 리스트를 value_count를 통하여 많이 중복된 순으로 곡을200개 가져온다.
                get_song = list(pd.value_counts(get_song)[:200].index)
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                
                ##중복을 제거한 후 100개까지 가져온다.
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
                #예외가 발생할 경우 results 데이터의 곡과 태그들을 집어넣는다
                #여기서 발생할 수 있는 예외는 곡이 0개일때를 뜻한다.
                #그러므로 여기로 오면 문제가 있는거임 (곡이있는 리스트만 불러왔기때문)
            except:
                error = error+1
                
                answers.append({
                  "id":most_results[most_results["id"]==q["id"]]["id"].values[0],
                  "songs": most_results[most_results["id"]==q["id"]]["songs"].values[0],
                  "tags":most_results[most_results["id"]==q["id"]]["tags"].values[0],
                }) 
        # 여기서 만약 중복제외곡이 100곡이 안된다면 most_results에서 곡을 더 가져와서 100개를 맞춘다.
        #태그도 마찬가지
        error3 = 0
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                ##곡이 100곡이 안됬을경우더하는값
                error3 = error3+1
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[most_results["id"]==q["id"]]["songs"].values[0])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[most_results["id"]==q["id"]]["tags"].values[0])[:10-len(q['tags'])]
        print(str(error)+"결과오류발생횟수")
        print(str(error3)+"곡 오류발생횟수(100곡이안되었을경우)")
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val,self.test)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val,self.test, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.test)
        write_json(self.answers, 'embedding_model_tag.json')

        

In [ ]:
FILE_PATH = ''
U_space = PlaylistEmbedding(FILE_PATH)
U_space.run()


### 곡, 태그 ,제목 토크나이즈 한것을 임베딩하는 모델

In [ ]:
class PlaylistEmbedding:
    ## init 에서 train , val 데이터를 주고 , Word2vec사용시 선택값들을 넣어준다.
    def __init__(self, FILE_PATH):
        with open(os.path.join(FILE_PATH, 'train_token.json'), encoding="utf-8") as f:
            self.train = json.load(f)
        with open(os.path.join(FILE_PATH, 'val_token.json'), encoding="utf-8") as f:
            self.val = json.load(f)
        with open(os.path.join(FILE_PATH, 'test_token.json'), encoding="utf-8") as f:
            self.test = json.load(f)
        self.most_results = pd.read_json("arena_data/most_popular.json")


        self.FILE_PATH = FILE_PATH  #('arena_data/')
        self.min_count = 2
        self.size = 128
        self.window = 210
        self.sg = 5
        self.p2v_model = WordEmbeddingsKeyedVectors(self.size)
       
        
        
      
            
    ## train 데이터와 val 데이터를 합친 후 그 플레이리스트의 id값에 곡들을 매칭시킨다.
  
    def get_dic(self, train, val,test):
        song_dic = {}
        tag_dic = {}
        data = train+val+test
        for q in tqdm(data):
            song_dic[str(q['id'])] = q['songs']
            tag_dic[str(q['id'])] = q['tags']
        self.song_dic = song_dic
        self.tag_dic = tag_dic
        ## 플레이리스트마다 곡과 태그들을 합쳐 리스트화 하는데 합친 값이 0 즉 곡과 태그들이 전혀없는경우는 제외한다.
        total = list(map(lambda x: list(map(str, x['songs'])) + list(x['tags']), data))
        total = [x for x in total if len(x)>1]
        self.total = total
        
        #Word2vec메소드 실행 (곡과 태그를 함께 임베딩)
    def get_w2v(self, total, min_count, size, window, sg):
        print("학습시작")
        w2v_model = Word2Vec(total, min_count = min_count, size = size, window = window, sg = sg,workers=6)
        w2v_model.wv.save_word2vec_format('getsong_2_topn12_dimension128') # 모델 저장
#         loaded_model = KeyedVectors.load_word2vec_format("token_model_send_iter10")
#         w2v_model = loaded_model
        self.w2v_model = w2v_model
        print("학습완료")
    ## 
    def update_p2v(self, train, val,test, w2v_model):
        ID = []   
        vec = []
        for q in tqdm(train+val+test):
            tmp_vec = 0
            
            ##곡이 1개 이상인 경우에 곡과 태그의 백터값을 구한후 모두 더해 플레이리스트의 벡터,
            ##즉 플레이리스트의 백터를 구한다 (tmp_vec)
            ##여기서 나는 곡이 1개이상인  val플레이리스트기준으로 하였기때문에 왠만하면 if문 무조건넘어감
            for song in q['songs']+q['tags']:
                        
                    try:
                        
                        tmp_vec += w2v_model.wv.get_vector(str(song))
                        not_error.append(str(song))
                    except:
                        
                        error2.append(str(song))
                        pass
            if type(tmp_vec)!=int:
                ID.append(str(q['id']))    
                vec.append(tmp_vec)
       #구한 플레이리스트 백터리스트를 p2vmodel에 저장한다.
        self.p2v_model.add(ID, vec)
        print("업데이트 완료")
    
    ## 플레이리스트 유사도를 구한후 결과를 내는 메소드
    def get_result(self, p2v_model, song_dic, tag_dic, most_results, val):
        answers = []
        error = 0
        for n, q in tqdm(enumerate(val), total = len(val)):
            try:
                #문제플레이리스트와 가장 유사한 플레이리스트 200개의 ID를 뽑는다.

                most_id = [x[0] for x in p2v_model.most_similar(str(q['id']), topn=12)]

                get_song = []
                get_tag = []
                #플레이리스트ID를 이용하여 안에 들어있는 곡과 태그를 전부 꺼낸다.
                for ID in most_id:
                    get_song += song_dic[ID]
                    get_tag += tag_dic[ID]
                 
                #꺼내진 리스트를 value_count를 통하여 많이 중복된 순으로 곡을200개 가져온다.
                get_song = list(pd.value_counts(get_song)[:200].index)
#                 val_update_time = q["updt_date"]
#                 get_song_update = []
#                 val_date = int(val_update_time.split("-")[0]+val_update_time.split("-")[1]+val_update_time.split("-")[2].split(" ")[0])
#                 for songs in get_song:
#                     song_date = song_meta[song_meta["id"]==songs]["issue_date"].values[0]
#                     if val_date-song_date>0:
#                         get_song_update.append(songs)
#                     else:
#                         print("날짜비정상")
#                         print(val_update_time+"=>"+str(val_date)+"-"+str(song_date))
                get_tag = list(pd.value_counts(get_tag)[:20].index)
                
                ##중복을 제거한 후 100개까지 가져온다.
                answers.append({
                    "id": q["id"],
                    "songs": remove_seen(q["songs"], get_song)[:100],
                    "tags": remove_seen(q["tags"], get_tag)[:10],
                })
                #예외가 발생할 경우 results 데이터의 곡과 태그들을 집어넣는다
                #여기서 발생할 수 있는 예외는 곡이 0개일때를 뜻한다.
                #그러므로 여기로 오면 문제가 있는거임 (곡이있는 리스트만 불러왔기때문)
            except:
                error = error+1
                
                answers.append({
                  "id":most_results[most_results["id"]==q["id"]]["id"].values[0],
                  "songs": most_results[most_results["id"]==q["id"]]["songs"].values[0],
                  "tags":most_results[most_results["id"]==q["id"]]["tags"].values[0],
                }) 
        # 여기서 만약 중복제외곡이 100곡이 안된다면 most_results에서 곡을 더 가져와서 100개를 맞춘다.
        #태그도 마찬가지
        error3 = 0
        for n, q in enumerate(answers):
            if len(q['songs'])!=100:
                ##곡이 100곡이 안됬을경우더하는값
                error3 = error3+1
                answers[n]['songs'] += remove_seen(q['songs'], self.most_results[most_results["id"]==q["id"]]["songs"].values[0])[:100-len(q['songs'])]
            if len(q['tags'])!=10:
                answers[n]['tags'] += remove_seen(q['tags'], self.most_results[most_results["id"]==q["id"]]["tags"].values[0])[:10-len(q['tags'])]
        print(str(error)+"결과오류발생횟수")
        print(str(error3)+"곡 오류발생횟수(100곡이안되었을경우)")
        self.answers = answers
    
    def run(self):
        self.get_dic(self.train, self.val,self.test)
        self.get_w2v(self.total, self.min_count, self.size, self.window, self.sg)
        self.update_p2v(self.train, self.val,self.test, self.w2v_model)
        self.get_result(self.p2v_model, self.song_dic, self.tag_dic, self.most_results, self.test)
        write_json(self.answers, 'embedding_model_has_song.json')

        

In [ ]:
FILE_PATH = ''
U_space = PlaylistEmbedding(FILE_PATH)
U_space.run()



## 곡이 없는부분을 뽑기위한 모델

In [ ]:
# ★★★★★★★★★★★★★★★★★★★ 경로 체크 
train = pd.read_json('/content/drive/My Drive/arena_data/train.json') 
val = pd.read_json('/content/drive/My Drive/arena_data/val.json') 
with open(os.path.join('/content/drive/My Drive/arena_data/val.json'), encoding="utf-8") as f: 
  val_json = json.load(f)
with open(os.path.join('/content/drive/My Drive/arena_data/results.json'), encoding="utf-8") as f:
  most_results = json.load(f)
test = pd.read_json('/content/drive/My Drive/arena_data/test.json')
with open(os.path.join('/content/drive/My Drive/arena_data/test.json'), encoding="utf-8") as f:
  test_json = json.load(f)
song_meta = pd.read_json('/content/drive/My Drive/arena_data/song_meta.json')

In [ ]:
# word_book = 데이터 프레임 을 만드는 과정 - 약 15분 소요
# word_book 은 데이터셋(현재코드는 트레인)에서 
# word_book = 단어 | 단어출현회수 | 단어가 출현한 플레이리스트 | 플레이리스트의 개수 | 곡(곡넘버 , 곡출현횟수 , 해당 단어의 곡이 출현할 퍼센트)

# 태그는 배열로 저장되어있어서 태그컬럼을 스트링으로 만들어 주는 포문
for i in train.index:
  tmp_str = ""
  for k in train.loc[i,'tags']:
    tmp_str = tmp_str + " " + k
  train.loc[i,'tags_token'] = tmp_str

# 트레인데이터의 제목 과 태그를 정제 후 스트링 
train['plylst_title'] = re_sub(train['plylst_title'])
train.loc[:, 'ply_token'] = train['plylst_title'].map(lambda x: get_token(x, tokenizer))

train['tags_token'] = re_sub(train['tags_token'])
train.loc[:, 'tags_token'] = train['tags_token'].map(lambda x: get_token(x, tokenizer))

using_pos = ['NNG','SL','NNP','XR']  # 일반 명사, 외국어, 고유 명사, 어근  
no_use_items = ['노래' , '플레이리스트' , '음악' ,'주관','요즘', '모음' , '나를' , '나만' ,'나에' , '내맘', '가요' , '초급' , '함께' , '너무' , '좋은' , '테스트' ,'가장' , '최고' , '명곡'
,'진짜' , '감성' , '매력' , '수록곡' , '그냥' , '없이' , '후기' , '느낌' , '오프닝' , '분위기', '삽입곡' ,'강추' ,'일때', '발매곡' , '입문' , '다재다능' ,'부제하','세대'
'자외' , '비트' , '고고씽' , '디편','사이','특별','순간','완전','BUT','세상','자국','저릿','THE','리메이','얘기','개인','선곡','코로','추천곡','대변','사람','멜로디'
'방법', '질때' , '분위기','정규앨범','가사','아실테죠','입장','위주','극복','사람','연습곡','은곡' , '컬렉션','TOP','베스트','드루','모두','이번','빠돌이','프로듀서','타이틀','고음불'
'다양','장르','소장','주간','마무리','메들리','월간','종합선물세트','결산','고기구' ,'느낌나','확찐','확빠', '필요' ,'집순']
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
train['ply_token'] = train['ply_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
train['tags_token'] = train['tags_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
train['tags_token'] = train['tags_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
train_token = train[['tags_token','ply_token']].values.tolist()

train_tags_title_list = [v  for v in train_token]
result_token_items = [] #하나의 리스트에 태그랑 토큰된제목이 들어가 있는 배열
for i , v in enumerate(train_tags_title_list):
  result_token_items.append([])
  for k in v:
    for j in k:
      if type(j) == str:
        result_token_items[i].append(j)
      else: # 데이터 타입이 튜플일 경우를 엘스
        result_token_items[i].append(j[0])

convert_str_token_list = []
for i , v in enumerate(result_token_items):
  convert_str_token_list.append([])
  append_word = ""
  for k in v:
    append_word = append_word +" "+str(k)
  convert_str_token_list[i].append(append_word)

#단어목록(단어 , 등장횟수)로 만들고 리스트로 변환후 데이터프레임에 word_book에 추가함
tag_title_token_list = [element for array in result_token_items for element in array]
token_temp = Counter(tag_title_token_list)
tokens, cnt = list(token_temp.keys()), list(token_temp.values())

# ★ 단어장으로 사용할 단어 데이터프레임
word_book = pd.DataFrame({'word':tokens , 'word_cnt':cnt})
word_book['playlist'] = np.empty((len(word_book), 0)).tolist() #플레이리스트에 빈배열로 넣어놓고 내용물은 추가하는방식으로 사용
word_book['playlist_cnt'] = 0 

# 각 단어가 어디 플레이리스트에 맵핑 되어잇는지 확인하는 코드
token_dic_with_playlist = {} 

playlist_title_dic_with_id = {} 
# 플레이리스트 아이디에 플레이리스트 스트링을 매핑 시켜놓았음 find문으로 단어가 포함되어있는지 아닌지를 확인할 예정
for i in train.index:
  playlist_title_dic_with_id[train.loc[i,'id']]=train.loc[i,'plylst_title']

tmp_token_with_playlist = {} # 단어 : 플레이리스트 목록 을 키로 만들어놓고 word_book에 매핑하도록함
for i in tqdm(tokens):
  tmp_token_with_playlist[i] = []
  for (key, elem) in playlist_title_dic_with_id.items():
    if i in elem:
      tmp_token_with_playlist[i].append(key)

print("word_book 데이터프레임 추가하는 작업중..")
word_book["playlist"] = word_book["word"].map(lambda x : tmp_token_with_playlist[str(x)])
tmp_cnt = [len(word_book.loc[i]['playlist']) for i in word_book.index]
word_book["playlist_cnt"] = tmp_cnt



In [ ]:
# word_book['songs_cnt_appear_percent']을 추가하는 코드
# 키값으론 단어(index)기준으로 (곡넘버 , 곡 출현횟수 , 단어에 곡이 출현한 확률)이 저장되어잇다
# 곡이 30프로 이상 출현 하면 정답으로 제출되며 추가로 벡터값(w2v)으로 시밀리러티한 곡들도 추가로 가져온다

songs_dic ={} #플레이리스트 아이디: 곡이랑 맵핑
for i in train.index:
  songs_dic[train.loc[i,'id']] = train.loc[i,'songs']

word_with_songs =[] # word_book와 인덱스가 맞춰져있다. 단어의 순서에 맞게 단어가 출현했을떄 사용된 곡들의 목록들이 들어있다.
for i in word_book.index:
  word_with_songs.append([]) 
  for k in word_book.loc[i,'playlist']: #해당단어가 출현한 플레이리스트의 목록을 가져와서
    for j in songs_dic.get(k):  # 해당 플레이리스트에 사용된 곡들을 word_with_songs 에 집어넣는다 
      word_with_songs[i].append(j) # word_book의 크기에 인덱스를 맞춰놓앗다

tmp_song_list = [Counter(i) for i in word_with_songs] #모든곡들의 중복을제거하면서 출현한 숫자를 카운트한다.
songs_count_list = [] # 카운팅된 숫자를 담을 리스트
for i , v in enumerate(tmp_song_list):
  count = word_book.loc[i,'playlist_cnt']
  songs_count_list.append([])
  for key , value in v.most_common(): #카운팅된 곡들을 가장많은 오름차순으로 정렬한다
    songs_count_list[i].append((key , value , value/count)) #저장하는방식 곡아이디 , 곡이출현한횟수 , 해당단어에서 곡이 출현확률 

word_book['songs_cnt_appear_percent'] = songs_count_list

# 플레이리스트가 2번이상 사용된 단어만 사용하기 위해서 데이터프레임을 하나 더 생성하고 
# 곡을 뽑을 시에 단어가 인풋으로 들어오면 곡의 출현 퍼센트를 이용해서 곡을 선택시 해당 filter_up2_word_book 이 사용된다. 
filter_up2_word_book = word_book[word_book['playlist_cnt'] > 1] 


In [ ]:
# 규정이 바뀌었으니 트레인, 발리데이션, 테스트 데이터셋에서 사용된 모든 곡만 임베딩을 진행한다
# 임베딩하는 이유는 곡을 뽑을떄 해당 단어가 일정퍼센트(30)이상 출현 할떄 퍼센트곡을 가져와서 사용하는데 
# 퍼센트 곡과 유사한곡을 구한후 정답에 추가 시키기 위해 사용된다.

mix_train_val_test = pd.concat([train, val, test] , ignore_index=True)
songs_for_embedding = []
for i in mix_train_val_test.index:
  songs_for_embedding.append([])
  for k in mix_train_val_test.loc[i,'songs']:
    songs_for_embedding[i].append(str(k))

w2v_model = Word2Vec(songs_for_embedding, min_count = 2, size=100, window=150, sg=1 , workers = multiprocessing.cpu_count())


In [ ]:
# 태그는 배열로 저장되어있어서 태그컬럼을 스트링으로 만들어 주는 포문
for i in test.index:
  tmp_str = ""
  for k in test.loc[i,'tags']:
    tmp_str = tmp_str + " " + k
  test.loc[i,'tags'] = tmp_str

# 발리데이션에 사용된 제목과 태그 가져오기
test['plylst_title'] = re_sub(test['plylst_title'])
test.loc[:, 'ply_token'] = test['plylst_title'].map(lambda x: get_token(x, tokenizer))
test['tags_token'] = re_sub(test['tags'])
test.loc[:, 'tags_token'] = test['tags'].map(lambda x: get_token(x, tokenizer))
using_pos = ['NNG','SL','NNP','XR']

test['ply_token'] = test['ply_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
test['ply_token'] = test['ply_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))

test['tags_token'] = test['tags_token'].map(lambda x: list(filter(lambda x: x[1] in using_pos, x)))
test['tags_token'] = test['tags_token'].map(lambda x: list(filter(lambda x: not(x[0]) in no_use_items, x)))
test_token = test[['tags_token','ply_token']].values.tolist()

test_tags_title_list = [v  for v in test_token]
test_token_items = [] 
# 하나의 리스트에 test-플레이리스트의 태그랑 토큰된 제목이 들어가 있는 배열
# 곡을 추출 하는 부분에서 단어를 이용해서 word_book에서 단어를 꺼내와야 하기때문에 문제와 인덱스를 맞추어서 단어를 저장하고 사용한다.
for i , v in enumerate(test_tags_title_list):
  test_token_items.append([])
  for k in v:
    for j in k:
      if type(j) == str:
        test_token_items[i].append(j)
      else: # 데이터 타입이 튜플일 경우를 엘스
        test_token_items[i].append(j[0])

In [ ]:
# 곡을 뽑아주는 코드블럭
# 1. 문제 플레이리스트에서 곡이 없는경우 (제목이 존재)에만 동작한다
# 2. test_token_items(태그 , 제목 의 형태소가 들어가 있어서) 해당 문제 id에 사용된 단어들을 포문으로 전달
# 2-1. 단어들이 filter_up2_word_book에서 곡이 출현확률이 30프로 이상이면 similary_song_list에 추가
# 2-2 30프로 이상 곡이 없을 수도 있으니 단어에 가장많이 사용된 곡들을 top_using_songs에 추가
# 3. top_using_songs이 곡들을 앞에서 30개만 남겨놓는다. 협업으로 곡을 넘겨줄떄 너무많으면 필요없기 떄문 
# 4. 2-1에서 추가된 곡이 있으면 w2v으로 임배딩한 곡들 중에서 30프로 이상곡과 유사한곡들을 가져와 similary_song_list에 추가하고 50곡만추림
# 5 출현확률 30프로이상 곡이 없는경우 즉 similary_song_list의 크기가 0인경우 top_using_songs을 similary_song_list에 추가
# 6. 예외처리용 코드 : 한번더 시밀러곡이 0이면 곡추가시켜줌
# 7. 수집한 곡중에서 문제 플레이리스트의 최종수정일보다 이전에 발매된 곡만 사용하기위해 필터링후 filter_issue_song에 추가
# 7-1. 예외처리용 코드 : filter_issue_song의 크기가 0이면 top_using_songs의 곡이라도 추가 시켜줌
# 8.중복된 곡을 제거하여 final_song_list에 추가 후 협업쪽으로 전달 

answers = []
get_tag = ["기분전환" , "발라드" , " 휴식" , "드라이브" , "감성" , "잔잔한" , "힐링" , "카페" , "힙합" , " 새벽"]

for n, q in tqdm(enumerate(test_json), total=len(test_json)):
  if len(q['songs']) == 0: # 곡이 존재하지 않는 경우
    similary_song_list = [] #단어에 따른 곡의 출현빈도가 특정수치 이상인 단어들을 모으는 배열
    top_using_songs = []  # 많이 사용된 단어 목록을 담아놓고 모자른곡 채워넣음
    similary_embedding_songs = [] #워드임베딩해놓은 곡중에서 유사한곡 가져오기
    filter_issue_song = [] #문제의 최종수정일보다 이후에있는 발매일을 가진 곡은 제외
    final_song_list = [] #곡 최종제출용 
    
    test_update_time = q['updt_date'] 

    for word in test_token_items[n]: 
      # 발리데이션에서 제목(토큰)과 태그를 가지고있을떄 해당 단어들을 포함하고있는 플레이리스트에서 곡이 출현확률이 0.3이상인거만 가져온다.
      try: #  단어가 단어장 filter_word_book에 존재 하지 않을경우를 예외처리
        targeted_songs = flatten(filter_up2_word_book.loc[filter_up2_word_book["word"] == word,'songs_cnt_appear_percent'])
        targeted_songs = [i[0] for i in targeted_songs if i[2] > 0.3]
        similary_song_list.extend(targeted_songs)
      except:
        pass
      top_using_songs.append(word_book[word_book["word"]== word]['playlist'].tolist())

    top_using_songs = flatten(top_using_songs) 
    top_using_songs = flatten(top_using_songs)
    top_using_songs = [songs_dic.get(i) for i in top_using_songs]
    top_using_songs = flatten(top_using_songs) 
    # 협업필터링에 사용될 30개만 넘겨주기로함
    top_using_songs = list(pd.value_counts(top_using_songs)[:30].index) 

    # filter_up2_word_book에서 단어당 30프로 이상 출현확률을 가지는 곡들의 목록이 존재 하는 경우 곡만 w2v으로 임베딩한 곡과
    # 30프로 이상 출현곡의 유사한 곡을 더 가져옴 
    
    if len(similary_song_list) > 0:
      for i in similary_song_list:
        try: #단일곡 나오면 임배딩안되있어서 에러처리
          similary_embedding_songs.append(w2v_model.wv.most_similar(str(i)))
        except:
          pass
      similary_embedding_songs = flatten(similary_embedding_songs)
      similary_embedding_songs = [int(i[0]) for i in similary_embedding_songs]
      similary_song_list.extend(similary_embedding_songs) #임배딩으로 가져온 곡도 추가시켜줌 
      if len(similary_song_list) > 50: # 협업에 너무많이 50곡만 전달하기로 했으니 50곡이상일 경우 앞에서부터 곡을 잘라냄
        similary_song_list = similary_song_list[:50] 

    else:
      similary_song_list.extend(top_using_songs) #퍼센트 30프로 이상 곡이없으면 중복곡 30개를 넣어준다.

    if len(similary_song_list) == 0: #예외처리용 코드 : 한번더 시밀러곡이 0이면 곡추가시켜줌
      similary_song_list.extend(top_using_songs)

    for i in similary_song_list: 
      # 수집한 곡중에서 문제 플레이리스트의 최종수정일보다 이전에 발매된 곡만 사용하기위해 필터링하는코드
      song_date = song_meta[song_meta["id"]==i]["issue_date"].values[0]
      test_date = int(test_update_time.split("-")[0]+test_update_time.split("-")[1]+test_update_time.split("-")[2].split(" ")[0])
      if test_date-song_date>0:
        filter_issue_song.append(i)

    # 발생하진 않을거 같은데 최종제출에 혹시 몰라서 아래 코드블럭은 예외처리를 위해 해놓음
    # 발매일 필터링으로 인한 제출용 곡이 하나도 없을 경우에는 탑 레이팅곡이라도 채워서 협업필터링에 전달한다.
    if len(filter_issue_song) == 0:
      filter_issue_song.extend(top_using_songs)

    for v in filter_issue_song: #중복된 곡을 제거하여 final_song_list에 추가 하는 코드
      if v not in final_song_list:
        final_song_list.append(v)

    final_song_list = final_song_list[:100]

    answers.append({"id": q['id'], "songs": final_song_list, "tags": get_tag}) 
  else:
    answers.append({"id": q['id'],"songs": most_results[n]['songs'],"tags": get_tag})


In [ ]:

# ★★ 경로 수정 필요 ★★
write_json(answers, 'nosong_result.json')


## 코사인 유사도를 통한 곡을 뽑는 모델

### 곡정보가 없는 부분,있는부분 두가지로 나누어 가져옴

In [ ]:
##실제제출용은 TEST.JSON이기때문에 바꿈
val =test

no_song_val = pd.read_json("arena_data/nosong_result.json")
no_song_val = no_song_val[no_song_val["id"].isin(val[val["songs"].map(len)==0]["id"].values)]
has_song_val = val[val["songs"].map(len)>0]


In [ ]:
def mapping_function(data, col1, col2): 
    # 플레이리스트 아이디(col1)와 수록곡(col2) 추출
    plylst_song_map = data[[col1, col2]]

    # unnest col2
    plylst_song_map_unnest = np.dstack(
        (
            np.repeat(plylst_song_map[col1].values, list(map(len, plylst_song_map[col2]))), 
            np.concatenate(plylst_song_map[col2].values)
        )
    )

    # unnested 데이터프레임 생성 : plylst_song_map
    plylst_song_map = pd.DataFrame(data = plylst_song_map_unnest[0], columns = plylst_song_map.columns)
    plylst_song_map[col1] = plylst_song_map[col1].astype(str)
    plylst_song_map[col2] = plylst_song_map[col2].astype(str)

    # unnest 객체 제거
    del plylst_song_map_unnest
    return plylst_song_map

### 실제 플레이리스트에만 나온 곡들을 정렬하고,dictionary를 만든후, 매트릭스를만든다 

In [ ]:
playlst_song_map = mapping_function(train, 'id', 'songs')
agg = pd.DataFrame(playlst_song_map['songs'].value_counts()).reset_index()
agg.columns = ['곡', '플레이리스트내의 등장횟수']
song_meta['id'] = song_meta['id'].astype(str)
agg = agg.merge(song_meta[['id', 'artist_name_basket', 'song_name', 'song_gn_gnr_basket']], how='left', left_on='곡', right_on='id')

In [ ]:
song_dic = {}
for i in tqdm(agg.index):
    id = agg.loc[i,"곡"]
    count = agg.loc[i,"플레이리스트내의 등장횟수"]
    song_dic[id] = count

In [ ]:
# 실제플레이리스트에만 나온 곡들만 가져오고 나오지 않은 곡들은 제외한다.
train["songs"] = train["songs"].map(lambda x: [ i  for i in  x if str(i) in song_dic])
val["songs"] = val["songs"].map(lambda x: [ i  for i in  x if str(i) in song_dic])
no_song_val["songs"] = no_song_val["songs"].map(lambda x: [ i  for i in  x if str(i) in song_dic])
has_song_val["songs"] = has_song_val["songs"].map(lambda x: [ i  for i in  x if str(i) in song_dic])

In [ ]:
# 학습플레이리스트에 들어있는 곡 개수를 따로 열 정보로 만들어준다.
train.loc[:,'num_songs'] = train['songs'].map(len)
# 활용할 플레이리스트 정보만 가져옴
plylst_train = train[['id','tags','plylst_title','songs','num_songs']]


#플레이리스트 -곡 매트릭스를 만들기위해 곡 id를 재 인덱싱한다
song_id_idx_dic = {}
song_idx_id_dic = {}
for k,i in enumerate(song_dic):
    song_id_idx_dic[str(k)] = i
    song_idx_id_dic[str(i)] = k

#플레이리스트데이터들에 재인덱싱 곡id를 넣어준다
plylst_train['songs'] = plylst_train['songs'].map(lambda x:[song_idx_id_dic[str(i)] for i in x] )
val['songs'] = val['songs'].map(lambda x:[song_idx_id_dic[str(i)] for i in x] )
has_song_val['songs'] = has_song_val['songs'].map(lambda x:[song_idx_id_dic[str(i)] for i in x] )
no_song_val['songs'] = no_song_val['songs'].map(lambda x:[song_idx_id_dic[str(i)] for i in x] )






In [ ]:
# 총 몇개의 곡이 들어있는지 저장 --> 매트릭스 data 정보
n_songs = sum(song_dic.values())

# 총 몇개의 플레이리스트가 있는지 저장 --> 매트릭스 row 정보
n_plylst = len(plylst_train)

# 곡 메타 정보에서 총 몇개의 곡이 들어있는지 저장 --> 매트릭스 col 정보
n_songs_all = len(song_dic)

# 매트릭스에 들어갈 값 추출 (data)
data = np.repeat(1, n_songs)

# row_position, col_position (data 값이 들어갈 position)
row_pos = np.repeat(range(n_plylst), plylst_train['num_songs'])
col_pos = [song for songs in plylst_train['songs'] for song in songs]

# 플레이 리스트 별로 어떤 곡이 들어가 있는지 표현해주는 행렬 생성
plylst_song_matrix = spr.csr_matrix((data, (row_pos, col_pos)), shape=(n_plylst, n_songs_all))
### 필요한 function
print(plylst_song_matrix.shape)

### 곡이 있을때 코사인유사도를 구하는부분

In [ ]:
# 문제 플레이리스트 하나에 대한 매트릭스 만들어주는 함수
def has_song_plylst_to_matrix(val_song_list, n_songs_all):
    # 플레이리스트가 가지고 있는 곡의 수
    n_songs = len(val_song_list)
    
    # 매트릭스에 들어갈 값 추출 (data)
    data = np.repeat(1, n_songs)
    
    # row_position, col_position (data 값이 들어갈 position)
    row_pos = np.repeat(0, n_songs)
    col_pos = [song for song in val_song_list]
    
    # 플레이 리스트 별로 어떤 곡이 들어가 있는지 표현해주는 행렬 생성
    val_song_matrix = spr.csr_matrix((data, (row_pos, col_pos)), shape=(1, n_songs_all))
    
    return val_song_matrix

# 플레이리스트 유사도 구해서 가장 비슷한 플레이리스트 뽑아오는 함수
def has_song_get_similar_plylsts_dict(val_song_matrix, plylst_song_matrix, top_n=10):
    
    # 플레이 리스트 별 유사도 구함
    plylst_sim = cosine_similarity(val_song_matrix, plylst_song_matrix)
    
    # 유사도가 높은 순으로 플레이리스트 index 정보 가져옴
    plylst_sim_sorted_ind = plylst_sim.argsort()[:, ::-1]
    similar_indexes = plylst_sim_sorted_ind[0, :(top_n)]
    
    # key : index, value : 유사도
    similar_plylst_dict = {x: round(plylst_sim[0,x],3) for x in similar_indexes if round(plylst_sim[0,x],3)}
    
    return similar_plylst_dict

# 결과로 제출할 곡 뽑는 함수 listing
def has_song_get_result_songs(val_song_list,real_val_song_list, similar_plylst_dict, plylst_song_matrix):
    
    # 매트릭스에서 플레이리스트의 인덱스값을 의미하는 key값을 담을 리스트
    key_list = [key for key in similar_plylst_dict if len(similar_plylst_dict) > 0]
    
    # 키 리스트에 해당하는 행의 매트릭스 정보를 표로 만듬
    df =pd.DataFrame(plylst_song_matrix[key_list].todense())
    
    
    ##오래걸리는부분
    # 인덱스값과 키값을 차례로 뽑아서
    for i,key in enumerate(similar_plylst_dict):
        # 유사도를 뽑음
        
        value = similar_plylst_dict[key]
        
        # 해당 플레이리스트 행에 유사도를 곱함 (곡 별로 가중치를 주는 것)
        df[i] = df[i] * value
        
    # 열별로 유사도 전부 더해서 높은 순으로 후보 곡을 200개 뽑음
    cend_song = df.sum().sort_values(ascending=False)[:200]
    
    ##### 곡id를 원래대로바꿈
    
        
    
    # 결과로 담을 최종 곡 리스트
    result_song = []
    
    # 원래 가지고 있는 곡 제외하고 100개를 담음
    for index in cend_song.index:
        
        if index not in real_val_song_list:
            if index not in val_song_list:
                result_song.append(index)
        if len(result_song) >= 100:
            break
            #원래 곡을 바궈주는 과정
    result_song = [song_id_idx_dic[str(i)] for i in result_song]
    if len(result_song) < 100:
        # 협업필터링만으로 곡이 전부 안채워졌을 경우 인기곡 담는다.
        require = 100 - len(result_song)
        
        song_list_added_cend = [song for song in song_mp if song not in result_song]
        song_list_added = [song for song in song_list_added_cend if song not in val_song_list]
        
        result_song = result_song + song_list_added[:require]
        if len(result_song) != 100:
            print('코드 다시봐라')
        
    return result_song

In [ ]:
val_plylst_id_dict = {x:y for (x,y) in enumerate(has_song_val['id'])}
val_plylst_songs_dict = {x:y for (x,y) in enumerate(has_song_val['songs'])}
real_val_plylst_songs_dict={x:y for (x,y) in enumerate(val['songs'])}

In [ ]:
res = []
result_dict = {}
n_get_plylst_all = 0
n_val_plylst_all = len(val)

for index in tqdm(range(len(val))):
    
    # 문제 플레이리스트 id 가져옴
    val_plylst_id = val_plylst_id_dict[index]
    # 그 플레이리스트가 가지고 있는 곡정보 가져옴
    val_already_songs = val_plylst_songs_dict[index]
    real_val_already_songs = real_val_plylst_songs_dict[index]
    # 문제 플레이리스트로 매트릭스 생성
    val_song_matrix = has_song_plylst_to_matrix(val_already_songs, n_songs_all)
    # 문제 플레이리스트와 가장 비슷한 플레이리스트들을 딕셔너리 형태로 가져옴
    similar_plylst_dict = has_song_get_similar_plylsts_dict(val_song_matrix, plylst_song_matrix)
    # 가져온 플레이리스트의 개수
    get_plylist_count = len(similar_plylst_dict)
    # 가져온 플레이리스트 개수 총 합 업데이트
    n_get_plylst_all = n_get_plylst_all + get_plylist_count
    # 문제 플레이리스트 id 별로 가져온 플레이리스트 개수 저장
    result_dict[val_plylst_id] = get_plylist_count
    # 곡 뽑아옴
    result_song = has_song_get_result_songs(val_already_songs,real_val_already_songs, similar_plylst_dict, plylst_song_matrix)
    # 결과에 추가
    res.append({
        "id": val_plylst_id,
        "songs": result_song,
        "tags": ["기분전환","발라드","감성","휴식","드라이브","힐링","새벽","잔잔한","사랑","카페"]
    })

print('----------------------------------------------')
print('평균적으로 가져오는 플레이리스트 개수 : ', n_get_plylst_all /  n_val_plylst_all)

In [ ]:
write_json(res, "cosine_model_has_song.json")

### 곡이 없을때 코사인 유사도를 구하는 부분

In [ ]:
# 문제 플레이리스트 하나에 대한 매트릭스 만들어주는 함수
def no_song_plylst_to_matrix(val_song_list, n_songs_all):
    # 플레이리스트가 가지고 있는 곡의 수
    n_songs = len(val_song_list)
    
    # 매트릭스에 들어갈 값 추출 (data)
    data = np.repeat(1, n_songs)
    
    # row_position, col_position (data 값이 들어갈 position)
    row_pos = np.repeat(0, n_songs)
    col_pos = [song for song in val_song_list]
    
    # 플레이 리스트 별로 어떤 곡이 들어가 있는지 표현해주는 행렬 생성
    val_song_matrix = spr.csr_matrix((data, (row_pos, col_pos)), shape=(1, n_songs_all))
    
    return val_song_matrix

# 플레이리스트 유사도 구해서 가장 비슷한 플레이리스트 뽑아오는 함수
def no_song_get_similar_plylsts_dict(val_song_matrix, plylst_song_matrix, top_n=10):
    
    # 플레이 리스트 별 유사도 구함
    plylst_sim = cosine_similarity(val_song_matrix, plylst_song_matrix)
    
    # 유사도가 높은 순으로 플레이리스트 index 정보 가져옴
    plylst_sim_sorted_ind = plylst_sim.argsort()[:, ::-1]
    similar_indexes = plylst_sim_sorted_ind[0, :(top_n)]
    
    # key : index, value : 유사도
    similar_plylst_dict = {x: round(plylst_sim[0,x],3) for x in similar_indexes if round(plylst_sim[0,x],3)}
    
    return similar_plylst_dict

# 결과로 제출할 곡 뽑는 함수 listing
def no_song_get_result_songs(val_song_list,real_val_song_list, similar_plylst_dict, plylst_song_matrix):
    
    # 매트릭스에서 플레이리스트의 인덱스값을 의미하는 key값을 담을 리스트
    key_list = [key for key in similar_plylst_dict if len(similar_plylst_dict) > 0]
    
    # 키 리스트에 해당하는 행의 매트릭스 정보를 표로 만듬
    df =pd.DataFrame(plylst_song_matrix[key_list].todense())
    
    
    ##오래걸리는부분
    # 인덱스값과 키값을 차례로 뽑아서
    for i,key in enumerate(similar_plylst_dict):
        # 유사도를 뽑음
        
        value = similar_plylst_dict[key]
        
        # 해당 플레이리스트 행에 유사도를 곱함 (곡 별로 가중치를 주는 것)
        df[i] = df[i] * value
        
    # 열별로 유사도 전부 더해서 높은 순으로 후보 곡을 200개 뽑음
    cend_song = df.sum().sort_values(ascending=False)[:200]
    
    ##### 곡id를 원래대로바꿈
    
        
    ##곡없을때는 유사곡을 구한후 밀어넣기를 하고 , 곡있을때는 그냥 카운트를 함

    # 결과로 담을 최종 곡 리스트
    result_song = val_song_list
    
    # 원래 가지고 있는 곡 제외하고 100개를 담음
    for index in cend_song.index:
        
        if index not in real_val_song_list:
            if index not in val_song_list:
                result_song.append(index)
        if len(result_song) >= 100:
            break
            #원래 곡을 바궈주는 과정
    result_song = [song_id_idx_dic[str(i)] for i in result_song]
    if len(result_song) < 100:
        # 협업필터링만으로 곡이 전부 안채워졌을 경우 인기곡 담는다.
        require = 100 - len(result_song)
        
        song_list_added_cend = [song for song in song_mp if song not in result_song]
        song_list_added = [song for song in song_list_added_cend if song not in val_song_list]
        
        result_song = result_song + song_list_added[:require]
        if len(result_song) != 100:
            print('코드 다시봐라')
        
    return result_song



In [ ]:
val_plylst_id_dict = {x:y for (x,y) in enumerate(no_song_val['id'])}
val_plylst_songs_dict = {x:y for (x,y) in enumerate(no_song_val['songs'])}
real_val_plylst_songs_dict={x:y for (x,y) in enumerate(val['songs'])}

In [ ]:
res = []
result_dict = {}
n_get_plylst_all = 0
n_val_plylst_all = len(val)

for index in tqdm(range(len(val))):
    
    # 문제 플레이리스트 id 가져옴
    val_plylst_id = val_plylst_id_dict[index]
    # 그 플레이리스트가 가지고 있는 곡정보 가져옴
    val_already_songs = val_plylst_songs_dict[index]
    real_val_already_songs = real_val_plylst_songs_dict[index]
    
    # 문제 플레이리스트로 매트릭스 생성
    val_song_matrix = no_song_plylst_to_matrix(val_already_songs, n_songs_all)
    # 문제 플레이리스트와 가장 비슷한 플레이리스트들을 딕셔너리 형태로 가져옴
    similar_plylst_dict = no_song_get_similar_plylsts_dict(val_song_matrix, plylst_song_matrix)
    # 가져온 플레이리스트의 개수
    get_plylist_count = len(similar_plylst_dict)
    # 가져온 플레이리스트 개수 총 합 업데이트
    
    
    n_get_plylst_all = n_get_plylst_all + get_plylist_count
    # 문제 플레이리스트 id 별로 가져온 플레이리스트 개수 저장
    result_dict[val_plylst_id] = get_plylist_count
    # 곡 뽑아옴
    result_song = no_song_get_result_songs(val_already_songs,real_val_already_songs, similar_plylst_dict, plylst_song_matrix)
    # 결과에 추가
    res.append({
        "id": val_plylst_id,
        "songs": result_song,
        "tags": ["기분전환","발라드","감성","휴식","드라이브","힐링","새벽","잔잔한","사랑","카페"]
    })

print('----------------------------------------------')
print('평균적으로 가져오는 플레이리스트 개수 : ', n_get_plylst_all /  n_val_plylst_all)

In [ ]:
write_json(res, "cosine_model_no_song.json")

## 기준에 맞춰 나온 모델들을 합치는 부분

In [ ]:
##주 모델 코사인유사도
cosine_model_has_song = pd.read_json("arena_data/cosine_model_has_song.json")
##곡없을때 뽑는 부분
cosine_model_no_song = pd.read_json("arena_data/cosine_model_no_song.json")
##유사도 0.15이하
embedding_model_has_song = pd.read_json("arena_data/embedding_model_has_song.json")
embedding_model_tag = pd.read_json("arena_data/embedding_model_tag.json")

### 임베딩으로 뽑은 곡+코사인유사도로 뽑은 곡 많이 겹친 곡 순으로 다시 재정렬 한후 가져온다

In [ ]:
## 모델 1과 2의 곡들을 비교하여 겹치는곡을 우선으로 병합하는 부분
res = []

for id in tqdm(cosine_model_has_song["id"]):
    #id에 맞는 리스트를 가져온다.
    mo1 = cosine_model_has_song[cosine_model_has_song["id"]==id]
    mo2 = embedding_model_has_song[embedding_model_has_song["id"]==id]
    ## 높은 ndcg를 기록하는 모델을 기준으로 병합을 함 (현재 model1이 높은 ndcg보유)
    merged_list = mo1.merge(mo2, on="id")
    
    
    
    #각각의 곡 리스트 합치기
    merged_song = merged_list["songs_x"]+merged_list["songs_y"]   
    songs_counter = Counter([tg for tgs in merged_song for tg in tgs])
    songs = []
    ## 합친 곡중 많이 겹친곡 순으로 100개 가져오기
    song_dict = {x: songs_counter[x] for x in songs_counter.most_common(100)}
    for i in song_dict:
        songs.append(i[0])


        
    #태그는 모델2의 태그만 가져오기때문에 X만 가져오면 됨 
    tags = merged_list["tags_y"]
    tag = []
    tags_counter = Counter([tg for tgs in tags for tg in tgs])
    for k in tags_counter:
        tag.append(k)
        
    res.append({
            "id": id,
                "songs": songs,
                "tags": tag
         })
    


In [ ]:
##합쳐진 18000개의 VAL 저장
write_json(res, "hassong.json")
has_song_model = pd.read_json("arena_data/hassong.json")

In [ ]:
list_has_song =  val[val["songs"].map(len)>0].values
list_not_song =  val[val["songs"].map(len)==0]["id"].values

In [ ]:
res = []
for idx in tqdm(list_has_song):
    songs = has_song_model[has_song_model["id"]==idx]["songs"].values[0]
    tag = embedding_model_tag[embedding_model_tag["id"]==idx]["tags"].values[0]
    
    res.append({
            "id": idx,
                "songs": songs,
                "tags": tag
         })
for idx in tqdm(list_not_song):
    songs = cosine_model_no_song[cosine_model_no_song["id"]==idx]["songs"].values[0]
    tag = embedding_model_tag[embedding_model_tag["id"]==idx]["tags"].values[0]
        
    res.append({
            "id": idx,
                "songs": songs,
                "tags": tag
         })

In [ ]:
len(res)


In [ ]:

write_json(res, "final_results.json")